# **Importing the libraries**

In [ ]:
import cv2 as cv 
import numpy as np
import math
import glob
import os

import matplotlib
from matplotlib import pyplot as plt

from skimage.metrics import structural_similarity
import imutils

# **Selecting the source folders**


*   Change the directory paths as needed before running
*   "directory" is the folder containing the test data (Example: evaluation/fake_test)
*   "resource_folder" is the folder containing the masks and other resources necessary for running the code ("Vasile_Andrei_408/resources")




In [ ]:
# The data source directory
directory = '/content/drive/MyDrive/Assignment/test'

# The folder containing the masks used
resource_folder = "/content/drive/MyDrive/Assignment"

# **Task 1**



## Target dartboard analysis

Obtain the number of darts and their locations (ex: 2,6,7)

(Dataset contains maximum 3 darts in a single image)

In [ ]:
# The template dartboard used as a model for aligning the other photos
model = (np.array(cv.imread(resource_folder + '/template_task1.jpg', flags = cv.IMREAD_GRAYSCALE)))
# The score mask used for determining the region the dart landed on
score_mask = (np.array(cv.imread(resource_folder + '/task 1 score mask.png', flags = cv.IMREAD_GRAYSCALE)))
# A mask of the exterior of the dart board used to remove the edges of the image where no darts could land
outer_mask = (np.array(cv.imread(resource_folder + '/task 1 outer mask.png', flags = cv.IMREAD_GRAYSCALE)))

directory_task1 = directory + '/Task1'
 

# Iterate over the files in directory
for filename in sorted(os.listdir(directory_task1)):
	path = os.path.join(directory_task1, filename)
	if os.path.isfile(path):
		target_image = (np.array(cv.imread(path, flags = cv.IMREAD_GRAYSCALE)))

		# Thresholding using Otsu's method
		ret,target_image_thresh = cv.threshold(target_image,127,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
		ret,model_thresh = cv.threshold(model,127,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

		# Applying Gaussian blur
		kernel_size = 5
		target_image_thresh = cv.GaussianBlur(target_image_thresh,(kernel_size, kernel_size),0)
		model_thresh = cv.GaussianBlur(model_thresh,(kernel_size, kernel_size),0)

		# Finding the keypoints in both images using ORB
		orb = cv.ORB_create(5000)
		(keypoints1, descriptors1) = orb.detectAndCompute(target_image_thresh, None)
		(keypoints2, descriptors2) = orb.detectAndCompute(model_thresh, None)

		# Matching the features
		method = cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
		matcher = cv.DescriptorMatcher_create(method)
		matches = matcher.match(descriptors1, descriptors2, None)

		matches = sorted(matches, key=lambda x:x.distance)

		# Keep only the top 10% of matches to reduce noise
		keep = int(len(matches) * 0.1)
		matches = matches[:keep]


		# Matching the keypoints in the image to the keypoints in the model
		points1 = np.zeros((len(matches), 2), dtype="float")
		points2 = np.zeros((len(matches), 2), dtype="float")
		
		for (i, m) in enumerate(matches):
				points1[i] = keypoints1[m.queryIdx].pt
				points2[i] = keypoints2[m.trainIdx].pt


		# Finding the homography
		matrix, mask = cv.findHomography(points1, points2, method=cv.RANSAC)

		(h, w) = model.shape[:2]

		# Aligning the images
		result = cv.warpPerspective(target_image, matrix, (w, h))


		result_copy = result.copy()
		model_copy = model.copy()

		result_copy = cv.GaussianBlur(result_copy,(25, 25),0)
		model_copy = cv.GaussianBlur(model_copy,(25, 25),0)


		# Now that the images are aligned finding the differences between them will give us an image of the darts
		(score, diff) = structural_similarity(result_copy, model_copy, full=True)
		diff = (diff * 255).astype("uint8")

		thresh = cv.threshold(diff, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]

		# The kernels used for dilating, opening and closing transformations
		kernel = cv.getStructuringElement(cv.MORPH_RECT, (10,1))
		kernel1 = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
		kernel2 = cv.getStructuringElement(cv.MORPH_RECT,(20,20))

		thresh_copy = thresh.copy()
		thresh_copy = cv.dilate(thresh_copy, kernel, iterations = 5)
		thresh_copy = cv.morphologyEx(thresh_copy, cv.MORPH_OPEN, kernel1)
		thresh_copy = cv.morphologyEx(thresh_copy, cv.MORPH_CLOSE, kernel2)

		# Removing the edges of the image to further remove noise
		thresh_copy = cv.subtract(thresh_copy, outer_mask)

		# Finding the countours
		contours = cv.findContours(thresh_copy.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
		contours = imutils.grab_contours(contours)
	

		# Taking the 6 biggest contours in case the darts got sectioned during the process of extracting the differences
		darts_countours = sorted(contours, key=cv.contourArea, reverse=True)[:6]


		# Finding the darts and the position they landed on
		darts_counter = 0
		darts_positions = []

		for dart in darts_countours:
			if(darts_counter == 3):
				break

			# We set a minimum contour area to avoid accepting left-over noise after processing
			if (cv.contourArea(dart) < 5667.6):
				pass

			# If the dart wasn't sectioned during processing it will have a larger area
			# In this case we just count the dart and check the location
			elif (cv.contourArea(dart) > 57667.6):
				extLeft = tuple(dart[dart[:, :, 0].argmin()][0])
				darts_counter = darts_counter + 1
				cv.circle(thresh_copy, extLeft, 8, (0, 0, 0), -1)
				darts_positions.append(score_mask[extLeft[1]][extLeft[0]])

			# Otherwise we try to find the tip of the dart by checking for
			# shapes with much larger width than height
			else:
				extLeft = tuple(dart[dart[:, :, 0].argmin()][0])
				extRight = tuple(dart[dart[:, :, 0].argmax()][0])
				extTop = tuple(dart[dart[:, :, 1].argmin()][0])
				extBot = tuple(dart[dart[:, :, 1].argmax()][0])

				width = abs(extLeft[0]-extRight[0])
				height = abs(extTop[1]-extBot[1])

				if (width > 2.4*height):
					darts_counter = darts_counter + 1
					cv.circle(thresh_copy, extLeft, 8, (0, 0, 0), -1)
					darts_positions.append(score_mask[extLeft[1]][extLeft[0]])



		# Writing the prediction text files
		answer_path = resource_folder + "/evaluation/submission_files/Vasile_Andrei_408/Task1/" + filename

		answer_txt = answer_path.replace(".jpg", "_predicted.txt")
		with open(answer_txt, 'w') as f:
			f.write(str(darts_counter)+"\n")
			for position in darts_positions:
				f.write(str(position)+"\n")

# **Task 2**

## Classic dartboard analysis

Obtain the number of darts and their locations (ex: s20, b25, t19)

(Dataset contains maximum 3 darts in a single image)

In [ ]:
# The template dartboard used as a model for aligning the other photos
model = (np.array(cv.imread(resource_folder + '/template_task2.jpg', flags = cv.IMREAD_GRAYSCALE)))
# The segment mask used for determining the region the dart landed
segment_mask = (np.array(cv.imread(resource_folder +  '/segments mask.png', flags = cv.IMREAD_GRAYSCALE)))
# The masks for the multiplier regions
single_ring_mask = (np.array(cv.imread(resource_folder +  '/single ring mask.png', flags = cv.IMREAD_GRAYSCALE)))
double_ring_mask = (np.array(cv.imread(resource_folder +  '/double ring mask.png', flags = cv.IMREAD_GRAYSCALE)))
triple_ring_mask = (np.array(cv.imread(resource_folder +  '/triple ring mask.png', flags = cv.IMREAD_GRAYSCALE)))
outer_b_mask = (np.array(cv.imread(resource_folder +  '/outer b mask.png', flags = cv.IMREAD_GRAYSCALE)))
inner_b_mask = (np.array(cv.imread(resource_folder +  '/inner b mask.png', flags = cv.IMREAD_GRAYSCALE)))
# The exterior of the dart board used to remove the edges of the image where no darts could land
outer_mask = (np.array(cv.imread(resource_folder +  '/task 1 outer mask.png', flags = cv.IMREAD_GRAYSCALE)))
ring = "s"

directory_task2 = directory + '/Task2'
 

# Iterate over the files in directory
for filename in sorted(os.listdir(directory_task2)):
	path = os.path.join(directory_task2, filename)
	if os.path.isfile(path):
		target_image = (np.array(cv.imread(path, flags = cv.IMREAD_GRAYSCALE)))

		# Thresholding using Otsu's method
		ret,target_image_thresh = cv.threshold(target_image,127,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
		ret,model_thresh = cv.threshold(model,127,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

		# Applying Gaussian blur
		kernel_size = 5
		target_image_thresh = cv.GaussianBlur(target_image_thresh,(kernel_size, kernel_size),0)
		model_thresh = cv.GaussianBlur(model_thresh,(kernel_size, kernel_size),0)

		# Finding the keypoints in both images using ORB
		orb = cv.ORB_create(5000)
		(keypoints1, descriptors1) = orb.detectAndCompute(target_image_thresh, None)
		(keypoints2, descriptors2) = orb.detectAndCompute(model_thresh, None)

		# Matching the features
		method = cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
		matcher = cv.DescriptorMatcher_create(method)
		matches = matcher.match(descriptors1, descriptors2, None)

		matches = sorted(matches, key=lambda x:x.distance)

		# Keep only the top 10% of matches to reduce noise
		keep = int(len(matches) * 0.1)
		matches = matches[:keep]


		# Matching the keypoints in the image to the keypoints in the model
		points1 = np.zeros((len(matches), 2), dtype="float")
		points2 = np.zeros((len(matches), 2), dtype="float")
		
		for (i, m) in enumerate(matches):
				# indicate that the two keypoints in the respective images
				# map to each other
				points1[i] = keypoints1[m.queryIdx].pt
				points2[i] = keypoints2[m.trainIdx].pt

		# Finding the homography
		matrix, mask = cv.findHomography(points1, points2, method=cv.RANSAC)

		(h, w) = model.shape[:2]

		# Aligning the images
		result = cv.warpPerspective(target_image, matrix, (w, h))

		result_copy = result.copy()
		model_copy = model.copy()

		result_copy = cv.GaussianBlur(result_copy,(35, 35),0)
		model_copy = cv.GaussianBlur(model_copy,(35, 35),0)

		# Now that the images are aligned finding the differences between them will give us an image of the darts
		(score, diff) = structural_similarity(result_copy, model_copy, full=True)
		diff = (diff * 255).astype("uint8")

		thresh = cv.threshold(diff, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]


		# The kernels used for dilating, opening and closing transformations
		kernel = cv.getStructuringElement(cv.MORPH_RECT, (10,1))
		kernel1 = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
		kernel2 = cv.getStructuringElement(cv.MORPH_RECT,(20,20))

		thresh_copy = thresh.copy()
		thresh_copy = cv.dilate(thresh_copy, kernel, iterations = 5)
		thresh_copy = cv.morphologyEx(thresh_copy, cv.MORPH_OPEN, kernel1)
		thresh_copy = cv.morphologyEx(thresh_copy, cv.MORPH_CLOSE, kernel2)

		# Removing the edges of the image to further remove noise
		thresh_copy = cv.subtract(thresh_copy, outer_mask)

		# Finding the countours
		contours = cv.findContours(thresh_copy.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
		contours = imutils.grab_contours(contours)


		# Taking the 6 biggest contours in case the darts got sectioned during the process of extracting the differences
		darts_countours = sorted(contours, key=cv.contourArea, reverse=True)[:6]

		# Finding the darts and the position they landed on
		darts_counter = 0
		darts_positions = []

		for dart in darts_countours:
			if(darts_counter == 3):
				break

			# We set a minimum contour area to avoid checking left-over noise after processing
			if (cv.contourArea(dart) < 3667.6):
				pass

			# If the dart wasn't sectioned during processing it will have a larger area
			# In this case we just count the dart and check the location
			elif (cv.contourArea(dart) > 37667.6):
				extLeft = tuple(dart[dart[:, :, 0].argmin()][0])
				darts_counter = darts_counter + 1
				cv.circle(thresh_copy, extLeft, 8, (0, 0, 0), -1)
				darts_positions.append(segment_mask[extLeft[1]][extLeft[0]])

			# Otherwise we try to find the tip of the dart by checking for
			# shapes with much larger width than height
			else:
				extLeft = tuple(dart[dart[:, :, 0].argmin()][0])
				extRight = tuple(dart[dart[:, :, 0].argmax()][0])
				extTop = tuple(dart[dart[:, :, 1].argmin()][0])
				extBot = tuple(dart[dart[:, :, 1].argmax()][0])

				width = abs(extLeft[0]-extRight[0])
				height = abs(extTop[1]-extBot[1])

				if (width > 2.4*height):
					darts_counter = darts_counter + 1
					cv.circle(thresh_copy, extLeft, 8, (0, 0, 0), -1)
					darts_positions.append(segment_mask[extLeft[1]][extLeft[0]])

					if (double_ring_mask[extLeft[1]][extLeft[0]] < 200):
					  ring = "d"
					elif (triple_ring_mask[extLeft[1]][extLeft[0]] < 200):
					  ring = "t"
					elif (outer_b_mask[extLeft[1]][extLeft[0]] < 200):
					  ring = "b"
					  darts_positions[-1] = 25
					elif (inner_b_mask[extLeft[1]][extLeft[0]] < 200):
					  ring = "b"
					  darts_positions[-1] = 50



		# Writing the prediction text files
		answer_path = resource_folder + "/evaluation/submission_files/Vasile_Andrei_408/Task2/" + filename

		answer_txt = answer_path.replace(".jpg", "_predicted.txt")

		with open(answer_txt, 'w') as f:
			f.write(str(darts_counter)+"\n")
			for position in darts_positions:
				f.write(ring + str(position)+"\n")

# **Task 3**

## Video analysis

Analysing videos where a single dart is thrown

Obtain the location of the last thrown dart


In [ ]:
model_3 = (np.array(cv.imread(resource_folder + '/auxiliary_images/template_task2.jpg', flags = cv.IMREAD_GRAYSCALE)))
score_mask = (np.array(cv.imread(resource_folder + '/task3_score3.png', flags = cv.IMREAD_GRAYSCALE)))

# The score masks for each template detected
score_mask1 = (np.array(cv.imread(resource_folder + '/task3_score1.png', flags = cv.IMREAD_GRAYSCALE)))
score_mask2 = (np.array(cv.imread(resource_folder + '/task3_score2.png', flags = cv.IMREAD_GRAYSCALE)))
score_mask3 = (np.array(cv.imread(resource_folder + '/task3_score3.png', flags = cv.IMREAD_GRAYSCALE)))

directory_task3 = directory + '/Task3'

# Iterate over the files in directory
for filename in sorted(os.listdir(directory_task3)):
	path = os.path.join(directory_task3, filename)
	if os.path.isfile(path):
		vidcap = cv.VideoCapture(path)
		success,image = vidcap.read()
		count = 0
		imagini = []
		while success:
			imagini.append(image)
			success,image = vidcap.read()
			count += 1

    # The dartboard templates
		template1 =(np.array(cv.imread("/content/drive/MyDrive/Assignment 3/task3_template3.png")))
		template2 =(np.array(cv.imread("/content/drive/MyDrive/Assignment 3/task3_template2.png")))
		template3 =(np.array(cv.imread("/content/drive/MyDrive/Assignment 3/task3_template1.png")))

		img_rgb = imagini[0].copy()
		img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)

		templates = []
		templates.append(cv.cvtColor(template1, cv.COLOR_BGR2GRAY)) 
		templates.append(cv.cvtColor(template2, cv.COLOR_BGR2GRAY)) 
		templates.append(cv.cvtColor(template3, cv.COLOR_BGR2GRAY)) 

		threshold = 0.8
		mask_number = 1

		for template in templates:
			w, h = template.shape[::-1]
			res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED)
			loc = np.where( res >= threshold)

      # If a template match has been found select the scoring mask for it
			if loc[0].size == 0:
				mask_number = mask_number + 1
			else:
				for pt in zip(*loc[::-1]):
					cv.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
				break
    
		if(mask_number == 1):
			score_mask = score_mask3
		elif(mask_number == 2):
			score_mask = score_mask2
		else:
			score_mask = score_mask1
      

		grayA = cv.cvtColor(imagini[0], cv.COLOR_BGR2GRAY)
		grayB = cv.cvtColor(imagini[-1], cv.COLOR_BGR2GRAY)
		(score, diff) = structural_similarity(grayA, grayB, full=True)
		diff = (diff * 255).astype("uint8")

		thresh_copy = cv.threshold(diff, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]

    # The kernels used for dilating, opening and closing transformations
		kernel = cv.getStructuringElement(cv.MORPH_RECT, (5,1))
		kernel1 = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
		kernel2 = cv.getStructuringElement(cv.MORPH_RECT,(15,15))

		thresh_copy = thresh_copy.copy()
		thresh_copy = cv.dilate(thresh_copy, kernel, iterations = 5)
		thresh_copy = cv.morphologyEx(thresh_copy, cv.MORPH_OPEN, kernel1)
		thresh_copy = cv.morphologyEx(thresh_copy, cv.MORPH_CLOSE, kernel2)

		contours = cv.findContours(thresh_copy.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
		contours = imutils.grab_contours(contours)


    # Taking the biggest contour
		dart = max(contours, key=cv.contourArea)

    # Finding the dart and the position it landed on
		darts_counter = 0
		dart_position = 0


		extLeft = tuple(dart[dart[:, :, 0].argmin()][0])
		darts_counter = darts_counter + 1
		cv.circle(thresh_copy, extLeft, 8, (0, 0, 0), -1)
		dart_position = score_mask[extLeft[1]][extLeft[0]]
		
		
    # The score masks follow the rule: 
    # single region = one and two digits pixel values (ex: 20, 5, 19)
    # double region = 3 digits values, 1 prefix (ex: 120, 105)
    # triple region = 3 digits values, 2 prefix

    # Selecting the multiplier region and dart position
		multiplier = dart_position 
		dart_position = dart_position % 100
		digits_number = 1

		# Extracting the first digit of the pixel value
		while (multiplier >= 10):
			multiplier = int(multiplier / 10)
			digits_number = digits_number + 1

		multiplier = multiplier + 1
		
		# Setting the letter for the multiplier
		if multiplier == 1 or digits_number < 3:
			letter = "s"
		elif multiplier == 2:
			letter = "d"
		else:
			letter = "t"

		if dart_position == 25 or dart_position == 50:
			letter = "b"

    # Writing the prediction text files
		answer_path = resource_folder + "/evaluation/submission_files/Vasile_Andrei_408/Task3/" + filename

		answer_txt = answer_path.replace(".mp4", "_predicted.txt")

		with open(answer_txt, 'w') as f:
			f.write(letter + str(dart_position))